In [2]:
from fastapi import FastAPI
from pydantic import BaseModel
import pandas as pd
import joblib
from typing import List

app = FastAPI()

# Load your trained model
model = joblib.load('loan_optimizer_model.joblib')

class LoanOption(BaseModel):
    min_amount: float
    interest_rate: float
    max_amount: float
    term_months: int
    loan_names: str

class UserProfile(BaseModel):
    total_assets_value: float
    average_monthly_income: float
    average_monthly_expenses: float
    target_savings: float
    time_horizon: int

@app.post("/evaluate-loans")
async def evaluate_loans(profile: UserProfile, loan_options: List[LoanOption]):
    # Convert loan options to DataFrame
    loan_df = pd.DataFrame([lo.dict() for lo in loan_options])
    
    # Process data as per your original script
    processed_df = loan_df.copy()
    processed_df['loan_amount'] = (processed_df['min_amount'] + processed_df['max_amount']) / 2
    monthly_surplus = max(profile.average_monthly_income - profile.average_monthly_expenses, 1)
    
    # Calculate derived features
    def calculate_payment(row):
        monthly_rate = row['interest_rate'] / 100 / 12
        loan_amount = row['loan_amount']
        term = row['term_months']
        if monthly_rate == 0:
            return loan_amount / term
        numerator = loan_amount * monthly_rate
        denominator = 1 - (1 + monthly_rate) ** -term
        return numerator / denominator
    
    processed_df['monthly_loan_payment'] = processed_df.apply(calculate_payment, axis=1)
    processed_df['total_repayment'] = processed_df['monthly_loan_payment'] * processed_df['term_months']
    processed_df['interest_cost'] = processed_df['total_repayment'] - processed_df['loan_amount']
    processed_df['loan_to_income'] = processed_df['loan_amount'] / profile.average_monthly_income
    processed_df['cash_runway'] = profile.total_assets_value / monthly_surplus

    # Create input DataFrame
    input_data = pd.DataFrame({
        'monthly_income': [profile.average_monthly_income] * len(processed_df),
        'monthly_expenses': [profile.average_monthly_expenses] * len(processed_df),
        'existing_assets': [profile.total_assets_value] * len(processed_df),
        'interest_rate': processed_df['interest_rate'],
        'loan_amount': processed_df['loan_amount'],
        'term_months': processed_df['term_months'],
        'time_horizon': [profile.time_horizon] * len(processed_df),
        'loan_names': processed_df['loan_names'],
        'loan_to_income': processed_df['loan_to_income'],
        'interest_cost': processed_df['interest_cost'],
        'cash_runway': processed_df['cash_runway']
    })

    # Make predictions
    processed_df['predicted_savings_margin'] = model.predict(input_data)
    
    # Sort and return results
    results = processed_df.sort_values('predicted_savings_margin', ascending=False)
    return results[['loan_names', 'loan_amount', 'term_months', 
                    'interest_rate', 'predicted_savings_margin']].to_dict(orient='records')

In [3]:
from fastapi import FastAPI
from pydantic import BaseModel
import pandas as pd
import joblib
from typing import List
import uvicorn
import nest_asyncio  # Needed for Jupyter

# Required for running async in Jupyter
nest_asyncio.apply()

app = FastAPI()

# ... [Keep your existing model and endpoint code here] ...

# Server configuration
PORT = 8000
HOST = "127.0.0.1"  # localhost

if __name__ == "__main__":
    print(f"Starting server on http://{HOST}:{PORT}")
    uvicorn.run(
        app,
        host=HOST,
        port=PORT,
        log_level="info",
        reload=False  # Disable reload in Jupyter
    )

INFO:     Started server process [15724]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


Starting server on http://127.0.0.1:8000
INFO:     127.0.0.1:50738 - "GET / HTTP/1.1" 404 Not Found
INFO:     127.0.0.1:50738 - "GET /favicon.ico HTTP/1.1" 404 Not Found
INFO:     127.0.0.1:50739 - "GET /evaluate-loans HTTP/1.1" 404 Not Found


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [15724]
